In [14]:
import re
import glob
from itertools import zip_longest

import fitz
import hierarchy

In [15]:
_reg = re.compile(r'(?!\d\s)([.])\s(?=[A-Z])')
def sentencize_dodf(s):
    sents = re.split(_reg, s)
    return [i[0] + (i[1] if not i[0].endswith('.') else '.') for i in
        zip_longest(sents[0::2], sents[1::2], fillvalue='.')]

def spaced_letters_fix(s):
    mts = re.split(r'((?:[A-ZÀ-Ž]{1,2}\s){3,})', s)
    offset = 0
    lis = [ s[:offset] ]
    for text, spaced in zip_longest(mts[0::2], mts[1::2], fillvalue=''):
        lis.append(text)
        lis.append(spaced.replace(' ', ''))
        lis.append(' ')
    lis.pop()    # last space is extra
    return ''.join(lis)



def drop_parenthesis(s):
    lis = re.split(r'([()])', s)
    ext_lis = lis + ['(']
    acc = 0
    new = []
    for tex, par in zip(ext_lis[0::2], ext_lis[1::2]):
        if tex:
            if not acc:
                new.append(tex)
        acc += (1 if par == '(' else -1)
    return ''.join(new)

def preprocess(s):
    s = drop_parenthesis(s)
    s = spaced_letters_fix(s)
    s = re.sub(r'\s{2,}', ' ', s)
    return s


In [16]:
files = glob.glob('pdf/*.pdf')
# files

In [17]:
doc_pdf = fitz.open(files[5])
h = hierarchy.mount_hierarchy(doc_pdf)
hierarchy.post_process_hierarchy(h);

In [18]:
s = h['SEÇÃO I'][0][1][0][1]
# s

In [19]:

for i in s[:-2]:
    print(i)
    print('\n')

DECRETO Nº 40.674, DE 02 DE MAIO DE 2020


Altera o Decreto nº 40.583, de 1º de abril de 2020, que dispõe sobre as medidas para 


enfrentamento da emergência de saúde pública de importância internacional decorrente do 


novo coronavírus.


O GOVERNADOR DO DISTRITO FEDERAL, no uso das atribuições que lhe conferem 


o artigo 100, incisos VII e XXVI, da Lei Orgânica do Distrito Federal, DECRETA:


Art. 1º O Decreto nº 40.583, de 1º de abril de 2020, passa a vigorar com a seguinte 


alteração:


“Art. 3º Ficam suspensos, no âmbito do Distrito Federal, até o dia 10 de maio de 2020:


.............................................” (NR)


Art. 2º Este Decreto entra em vigor na data de sua publicação.


Brasília, 02 de maio de 2020.




In [20]:
%%time
sentences = sentencize_dodf(i)
for sent in sentences:
    print(sent, end='\n\n\n')

Brasília, 02 de maio de 2020..


CPU times: user 2.39 ms, sys: 0 ns, total: 2.39 ms
Wall time: 1.85 ms


In [21]:
reg = r'[.:;]\s(?=[A-Z])'
l = [''.join(tup) for tup in
    zip_longest(re.split(reg, i), re.findall(reg, i), fillvalue='.')
]
for _ in l:
    print(_)
    print('----')

Brasília, 02 de maio de 2020..
----


In [22]:
dic=hierarchy.hierarchy_text(h)
del dic['SEÇÃO 0']
tt = []
for v in dic.values():
    tt.extend(v)
lis = [i for i in tt if not i == i.upper()]
no_break = [i.replace('\n', ' ') for i in lis]

In [23]:
sentencize_dodf(preprocess( no_break[2] ))

['novo coronavírus..']

In [24]:
sentencize_dodf(preprocess( no_break[3] ))

['O GOVERNADOR DO DISTRITO FEDERAL, no uso das atribuições que lhe conferem .']

In [25]:
no_break

['Altera o Decreto nº 40.583, de 1º de abril de 2020, que dispõe sobre as medidas para ',
 'enfrentamento da emergência de saúde pública de importância internacional decorrente do ',
 'novo coronavírus.',
 'O GOVERNADOR DO DISTRITO FEDERAL, no uso das atribuições que lhe conferem ',
 'o artigo 100, incisos VII e XXVI, da Lei Orgânica do Distrito Federal, DECRETA:',
 'Art. 1º O Decreto nº 40.583, de 1º de abril de 2020, passa a vigorar com a seguinte ',
 'alteração:',
 '“Art. 3º Ficam suspensos, no âmbito do Distrito Federal, até o dia 10 de maio de 2020:',
 'Art. 2º Este Decreto entra em vigor na data de sua publicação.',
 'Brasília, 02 de maio de 2020.',
 '132º da República e 61º de Brasília',
 'A Subsecretaria de Administração Geral – SUAG/SES comunica, por meio do Ofício N° ',
 '801/2020,  a  abertura  para  recebimento  de  propostas  referente  à',
 'Redação, Administração e Editoração:  Anexo do Palácio do Buriti, Sala 102, Térreo.  CEP: 70075-900, Brasília/DF.  Telefones: (0XX61